<a href="https://colab.research.google.com/github/bruAristimunha/polos-univesp/blob/master/Polos_Univesp_Raspagem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
m


In [0]:
# 0.5 doesn't support tooltip, must install dev instead
#!pip install git+https://github.com/python-visualization/folium

from google.colab import files
import pandas as pd
import urllib3 as req
import folium 
import json

http = req.PoolManager()

In [0]:
req.disable_warnings(req.exceptions.InsecureRequestWarning)

In [0]:
univesp = pd.read_csv("https://raw.githubusercontent.com/bruAristimunha/polos-univesp/master/univesp_address.csv")
univesp.columns = ['Nome', 'Endereco',  'Rótulo','Licenciaturas', 'Computação']

In [0]:
univesp['Endereco'] = univesp['Endereco'] +" - "+ univesp['Nome'] + " - SP" 

In [0]:
chave = input()

In [5]:
string = 'https://maps.googleapis.com/maps/api/geocode/json?address='

univesp['json'] = [0]*len(univesp['Endereco'])

for indice, endereco in enumerate(univesp['Endereco']):
  requisicao = (string+endereco.replace(' ','+')+chave).encode('ascii', 'ignore').decode('ascii')
  #print(requisicao,indice)
  r = http.request('GET', requisicao)
  univesp['json'][indice] = r.data

#https://maps.googleapis.com/maps/api/geocode/json?address=1600+Amphitheatre+Parkway,+Mountain+View,+CA&key=YOUR_API_KEY

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
def makeMark(ind, lat, long, m):
  folium.Marker(
    location=[lat, long],
    popup=str(ind),
  ).add_to(m)


In [0]:
lat = lambda c : pd.DataFrame.from_dict(pd.DataFrame.from_dict(c['results'])['geometry'][0]).T['lat']['location']
lng = lambda c : pd.DataFrame.from_dict(pd.DataFrame.from_dict(c['results'])['geometry'][0]).T['lng']['location']


In [0]:
b=univesp['json'][343]

buscar_mao = univesp[univesp['json'] == b]
filtrar = univesp[~(univesp['json'] == b)]
lista_filtro = [45, 90, 107]#45,176,264]


In [0]:
m = folium.Map(
    location=[-22.315480, -49.073370],
    zoom_start=7
)


In [0]:
for ind in lista_filtro:
  buscar_mao = buscar_mao.append(univesp.iloc[[ind]])
  filtrar = filtrar.drop(ind,0)



In [0]:
for ind, js in enumerate(filtrar['json']):
  makeMark (ind, (lat (json.loads(js))), lng(json.loads(js)),m)